In [1]:
import pandas as pd
import numpy as np
from cobra.io import read_sbml_model
from tqdm.auto import tqdm

In [2]:
from os import listdir
from os.path import isfile, join

mypath = '../models/non-ec/'

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)

['Bacteroides_thetaiotaomicron_VPI_5482.xml', 'Bacteroides_uniformis_ATCC_8492.xml', 'Coprococcus_comes_ATCC_27758.xml', 'Escherichia_coli_ED1a.xml', 'Fusobacterium_nucleatum_subsp_nucleatum_ATCC_25586.xml', 'Roseburia_intestinalis_L1_82.xml', 'Streptococcus_parasanguinis_ATCC_15912.xml', 'Streptococcus_salivarius_DSM_20560.xml', '.DS_Store', 'e_coli_core.xml.gz']


In [3]:
db_l = []

for file in tqdm(onlyfiles):
    file = mypath + file
    model = read_sbml_model(file)
    name = model.id
    name = str(name)
    name = name.strip('M_')
    id_n = name
    name = name.replace('_',' ')
    rxns = len(model.reactions)
    mets = len(model.metabolites)
    name_list = name.split(' ')
    genus = name_list[0]
    species = ' '.join(name_list[1::])
    for i in range(1,7):
        sam_n = i
        sample = f'sample_{sam_n}'
        abundace = np.random.randint(100,500)
        res = [id_n, genus, species, rxns, mets, file, sample, abundace]
        db_l.append(res)
    
db = pd.DataFrame(db_l, columns = ['id','genus','species','reactions','metabolites','file','sample_id','abundace'])
db

  0%|          | 0/10 [00:00<?, ?it/s]

Set parameter Username


CobraSBMLError: Something went wrong reading the SBML model. Most likely the SBML model is not valid. Please check that your model is valid using the `cobra.io.sbml.validate_sbml_model` function or via the online validator at https://sbml.org/validator_servlet/ .
	`(model, errors) = validate_sbml_model(filename)`
If the model is valid and cannot be read please open an issue at https://github.com/opencobra/cobrapy/issues .

In [ ]:
from micom.workflows import build

manifest = build(db, out_folder="../data/micom", model_db=None, cutoff=0.0001, threads=2)

In [ ]:
manifest

In [ ]:
from micom import load_pickle

com1 = load_pickle("../data/micom/sample_1.pickle")
print(len(com1.reactions))

In [ ]:
from micom.media import minimal_medium

med = minimal_medium(com1, 0.8)
res_m = []

for k in med.keys():
    print(f'Reaction: {k}, Flux: {med[k]}')
    rx = k
    flux = med[k]
    met = k.strip('EX_')
    res = [k,k,flux,met]
    res_m.append(res)
    
min_med = pd.DataFrame(res_m, columns = ['reaction','reaction','flux','metabolite'])
min_med

In [ ]:
from micom.data import test_medium
from micom.qiime_formats import load_qiime_medium

test_med = load_qiime_medium(test_medium)
test_med

In [ ]:
from micom.workflows import grow

res = grow(manifest, model_folder="../data/micom", medium=min_med, tradeoff=1.0, threads=2)

In [ ]:
from micom.workflows import tradeoff

tradeoff_rates = tradeoff(manifest, model_folder="../data/micom", medium=min_med, threads=2)
tradeoff_rates.groupby("tradeoff").apply(
    lambda df: (df.growth_rate > 1e-6).sum()).reset_index()

In [ ]:
from micom.viz import plot_tradeoff

pl = plot_tradeoff(tradeoff_rates, filename="../data/micom/micomtradeoff.html")

In [15]:
from micom.viz import plot_growth

pl = plot_growth(res, filename="../data/micom/micomgrowth_rates.html")

In [16]:
pl.view()